In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xml.etree.ElementTree as ET  
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from statistics import mean
import math
import os

In [40]:
def exctract_features(path):
  features={}

  tree = ET.parse(path)
  root = tree.getroot()
  #Print Description
  for x in root[0]:
    print(x.text)
  #Extraction of coordinates
  a=[]
  b=[]
  all_x=[]
  all_y=[]
  x_coordinates = []
  y_coordinates = []
  for i in range(1,len(root)):
    for x in root[i]:
      if (x.text=='\n\t\t\t'):
        continue
      else:
        l = list(map(str,x.text.split(',')))
        l = [x.strip() for x in l]
        for x in l:
          xy = list(map(int,x.split(' ')))
          x_coordinates.append(xy[0])
          y_coordinates.append(xy[1])
        a.append(x_coordinates)
        b.append(y_coordinates)
        x_coordinates = []
        y_coordinates = []  
  for i in range(len(a)):
    for j in range(len(a[i])):
      all_x.append(a[i][j])
      all_y.append(b[i][j])
  
  #Time Stamp
  time_stamp = np.arange(0,len(all_x)*0.02,0.02)
  
  #Relative_x_y
  relative_x=[0]
  relative_y=[0]
  for i in range(1,len(all_x)):
    relative_x.append(all_x[i]-mean(all_x[:i]))
    relative_y.append(all_y[i]-mean(all_y[:i]))
  features['relative_x']=relative_x
  features['realtive_y']=relative_y

  #Speed
  tracewise_speed=[0]
  speed=[]
  whole_speed=[]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      tracewise_speed.append(((a[i][j]-a[i][j-1])**2-(b[i][j]-b[i][j-1])**2)**(0.5)/0.02)
    speed.append(tracewise_speed)
    tracewise_speed=[0]
  for i in range(len(speed)):
    for j in range(len(speed[i])):
      whole_speed.append(speed[i][j])
  features['speed']=whole_speed

  #Speed along x and y
  speed_x=[]
  speed_y=[]
  tracewise_speed_x=[]
  tracewise_speed_y=[]
  temp_x=[0]
  temp_y=[0]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      temp_x.append((a[i][j]-a[i][j-1])/0.02)
      temp_y.append((b[i][j]-b[i][j-1])/0.02)
    tracewise_speed_x.append(temp_x)
    tracewise_speed_y.append(temp_y)
    temp_x=[0]
    temp_y=[0]
  for i in range(len(tracewise_speed_x)):
    for j in range(len(tracewise_speed_x[i])):
      speed_x.append(tracewise_speed_x[i][j])
      speed_y.append(tracewise_speed_y[i][j])
  features['speed_x'] = speed_x
  features['speed_y'] = speed_y

  #Acceleration
  acc=[]
  tracewise_acc=[0]
  whole_acc=[]
  for i in range(len(speed)):
    for j in range(1,len(speed[i])):
      tracewise_acc.append((speed[i][j]-speed[i][j-1])/0.02)
    acc.append(tracewise_acc)
    tracewise_acc=[0]
  for i in range(len(acc)):
    for j in range(len(acc[i])):
      whole_acc.append(acc[i][j])
  features['acceleration']=whole_acc

  #Acceleration along x and y
  acc_x=[0]
  acc_y=[0]
  tracewise_acceleration_x=[]
  tracewise_acceleration_y=[]
  acceleration_x=[]
  acceleration_y=[]
  for i in range(len(tracewise_speed_x)):
    for j in range(1,len(tracewise_speed_x[i])):
      acc_x.append((tracewise_speed_x[i][j]-tracewise_speed_x[i][j-1])/0.02)
      acc_y.append((tracewise_speed_y[i][j]-tracewise_speed_y[i][j-1])/0.02)
    tracewise_acceleration_x.append(acc_x)
    tracewise_acceleration_y.append(acc_y)
    acc_x=[0]
    acc_y=[0]
  for i in range(len(tracewise_acceleration_x)):
    for j in range(len(tracewise_acceleration_x[i])):
      acceleration_x.append(tracewise_acceleration_x[i][j])
      acceleration_y.append(tracewise_acceleration_y[i][j])
  features['acceleration_x']=acceleration_x
  features['acceleration_y']=acceleration_y

  #Log curvature radius
  rad=[]
  tracewise_rad=[]
  whole_rad=[]
  for i in range(len(tracewise_speed_x)):
    for j in range(len(tracewise_speed_x[i])):
      nr=(tracewise_speed_x[i][j]*tracewise_acceleration_y[i][j]-tracewise_acceleration_x[i][j]-tracewise_speed_y[i][j])
      dr=(tracewise_speed_x[i][j]**2+tracewise_speed_y[i][j]**2)**(1.5)
      if (nr==0 or dr==0):
        tracewise_rad.append(0)
      else:
        tracewise_rad.append(nr/dr)
    rad.append(tracewise_rad)
    tracewise_rad=[]
  for i in range(len(rad)):
    for j in range(len(rad[i])):
      whole_rad.append(rad[i][j])
  features['log_curvature_radius']=whole_rad

  #writing direction
  tracewise_cos=[0]
  tracewise_sin=[0]
  cos=[]
  sin=[]
  whole_cos=[]
  whole_sin=[]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      l=((a[i][j]-a[i][j-1])**2+(b[i][j]-b[i][j-1])**2)**(0.5)
      tracewise_cos.append((a[i][j]-a[i][j-1])/l)
      tracewise_sin.append((b[i][j]-b[i][j-1])/l)
    cos.append(tracewise_cos)
    sin.append(tracewise_sin)
    tracewise_cos=[0]
    tracewise_sin=[0]
  for i in range(len(cos)):
    for j in range(len(cos[i])):
      whole_cos.append(cos[i][j])
      whole_sin.append(sin[i][j])
  features['writing_direction_x']=whole_cos
  features['writing_direction_y']=whole_sin

  #Curvature
  cur_x=[]
  cur_y=[]
  tracewise_cur_x=[0]
  tracewise_cur_y=[0]
  whole_cur_x=[]
  whole_cur_y=[]
  for i in range(len(cos)):
    for j in range(1,len(cos[i])):
      tracewise_cur_x.append(np.cos(cos[i][j-1])*np.cos(cos[i][j])+np.sin(cos[i][j-1])*np.sin(cos[i][j]))
      tracewise_cur_y.append(np.cos(sin[i][j-1])*np.sin(sin[i][j])-np.sin(sin[i][j-1])*np.cos(sin[i][j]))
    cur_x.append(tracewise_cur_x)
    cur_y.append(tracewise_cur_y)
    tracewise_cur_x=[0]
    tracewise_cur_y=[0]
  for i in range(len(cur_x)):
    for j in range(len(cur_x[i])):
      whole_cur_x.append(cur_x[i][j])
      whole_cur_y.append(cur_y[i][j])
  features['curvature_x']=whole_cur_x
  features['curvature_y']=whole_cur_y

  #Vicinity Aspect
  temp_x=[]
  temp_y=[]
  left_x=[]
  right_x=[]
  left_y=[]
  right_y=[]
  var=[]
  vc=[]
  vl=[]
  vs_cos=[]
  vs_sin=[]
  tracewise_var=[]
  for i in range(len(a)):
    for j in range(len(a[i])):
      if len(a[i])>=11:
        if len(a[i][:j])>5:
          left_x=a[i][j-5:j]
        elif len(a[i][:j])<=5:
          left_x=a[i][:j]
        if len(a[i][j+1:])>5:
          right_x=a[i][j+1:j+6]
        elif len(a[i][j+1:])<=5:
          right_x=a[i][j+1:]
      elif len(a[i])<11:
        if len(a[i][:j])>5:
          left_x=a[i][j-5:j]
        elif len(a[i][:j])<=5:
          left_x=a[i][:j]
        if len(a[i][j+1:])>5:
          right_x=a[i][j+1:j+6]
        elif len(a[i][j+1:])<=5:
          right_x=a[i][j+1:]

      if len(b[i])>=11:
        if len(b[i][:j])>5:
          left_y=b[i][j-5:j]
        elif len(b[i][:j])<=5:
          left_y=b[i][:j]
        if len(b[i][j+1:])>5:
          right_y=b[i][j+1:j+6]
        elif len(b[i][j+1:])<=5:
          right_y=b[i][j+1:]
      elif len(b[i])<11:
        if len(b[i][:j])>5:
          left_y=b[i][j-5:j]
        elif len(b[i][:j])<=5:
          left_y=b[i][:j]
        if len(b[i][j+1:])>5:
          right_y=b[i][j+1:j+6]
        elif len(b[i][j+1:])<=5:
          right_y=b[i][j+1:]

      left_x.append(a[i][j])
      left_y.append(b[i][j])
      temp_x=left_x+right_x
      temp_y=left_y+right_y
      dx=max(temp_x)-min(temp_x)
      dy=max(temp_y)-min(temp_y)
      if dx+dy==0:
        var.append(0)
      else:
        var.append((dy-dx)/(dx+dy))
      
      #vicinity_curliness
      dist=[]
      for k in range(1,len(temp_x)):
        dist.append(((temp_x[k]-temp_x[k-1])**2+(temp_y[k]-temp_y[k-1])**2)**(0.5))
      if dx+dy==0:
        vc.append(0)
      else:
        vc.append(sum(dist)/max(dx,dy))
      dist=[]

      #vicinity linearity
      if temp_x[-1]==temp_x[0]:
        slope = 'vertical_line'
      else:
        slope = (temp_y[-1]-temp_y[0])/(temp_x[-1]-temp_x[0])
      dist=[]
      for k in range(len(temp_x)):
        if slope=='vertical_line':
          dist.append(0)
        else:
          c=temp_y[0]-slope*temp_x[0]
          dist.append(((abs(slope*temp_x[k]-1*temp_y[k]+c))/(slope**2+1)**(0.5))**2)
      
      vl.append(mean(dist))
      dist=[]

      #vicinity slope
      if (temp_x[-1]-temp_x[0])==0:
        vs_cos.append(1)
        vs_sin.append(0)
      else:
        slope = (temp_y[-1]-temp_y[0])/(temp_x[-1]-temp_x[0])
        theta = math.atan(slope)
        vs_cos.append(math.sin(theta))
        vs_sin.append(math.cos(theta))

  features['vicinity_aspect']=var
  features['vicinity_curliness']=vc
  features['vicinity_linearity']=vl
  features['vicinity_slope_x']=vs_cos
  features['vicinity_slope_y']=vs_sin

  return(features)